<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/Sbu-logo.svg/1200px-Sbu-logo.svg.png" alt="keras" width="150" height="150">

<h1 align=center><font size = 7>NLP Summer school</font></h1>
<h1 align=center><font size = 6>NLP Research Lab</font></h1>
<h1 align=center><font size = 5>Shahid Beheshti University</font></h1>
<h1 align=center><font size = 4> July 2022 </font></h1>

## Original Repo
https://github.com/olegborisovv/NGram_LanguageModel

With the great thanks to olegborisovv

In [ ]:
import string
import random
import time
from typing import List

In [ ]:
# ideally we would use some smart text tokenizer, but for simplicity use this one
def tokenize(text: str) -> List[str]:
    """
    :param text: Takes input sentence
    :return: tokenized sentence
    """
    for punct in string.punctuation:
        text = text.replace(punct, ' '+punct+' ')
    t = text.split()

    return t

In [ ]:
def get_ngrams(n: int, tokens: list) -> list:
    """
    :param n: n-gram size
    :param tokens: tokenized sentence
    :return: list of ngrams

    ngrams of tuple form: ((previous wordS!), target word)
    """
    # tokens.append('<END>')
    tokens = (n-1)*['<START>']+tokens
    l = [(tuple([tokens[i-p-1] for p in reversed(range(n-1))]), tokens[i]) for i in range(n-1, len(tokens))]
    return l

In [ ]:
class NgramModel(object):

    def __init__(self, n):
        self.n = n

        # dictionary that keeps list of candidate words given context
        self.context = {}

        # keeps track of how many times ngram has appeared in the text before
        self.ngram_counter = {}

    def update(self, sentence: str) -> None:
        """
        Updates Language Model
        :param sentence: input text
        """
        n = self.n
        ngrams = get_ngrams(n, tokenize(sentence))
        for ngram in ngrams:
            if ngram in self.ngram_counter:
                self.ngram_counter[ngram] += 1.0
            else:
                self.ngram_counter[ngram] = 1.0

            prev_words, target_word = ngram
            if prev_words in self.context:
                self.context[prev_words].append(target_word)
            else:
                self.context[prev_words] = [target_word]

    def prob(self, context, token):
        """
        Calculates probability of a candidate token to be generated given a context
        :return: conditional probability
        """
        try:
            count_of_token = self.ngram_counter[(context, token)]
            count_of_context = float(len(self.context[context]))
            result = count_of_token / count_of_context

        except KeyError:
            result = 0.0
        return result

    def random_token(self, context):
        """
        Given a context we "semi-randomly" select the next word to append in a sequence
        :param context:
        :return:
        """
        r = random.random()
        map_to_probs = {}
        token_of_interest = self.context[context]
        for token in token_of_interest:
            map_to_probs[token] = self.prob(context, token)

        summ = 0
        for token in sorted(map_to_probs):
#             return token
            summ += map_to_probs[token]
            if summ > r:
                return token

    def generate_text(self, token_count: int):
        """
        :param token_count: number of words to be produced
        :return: generated text
        """
        n = self.n
        context_queue = (n - 1) * ['<START>']
        result = []
        for _ in range(token_count):
            obj = self.random_token(tuple(context_queue))
            result.append(obj)
            if n > 1:
                context_queue.pop(0)
                if obj == '.':
                    context_queue = (n - 1) * ['<START>']
                else:
                    context_queue.append(obj)
        return ' '.join(result)

How can we add smoothing functionality?

Why didn't we return the most probable token? (stay tuned for the rest of the materials)






In [ ]:
def create_ngram_model(n, path):
    m = NgramModel(n)
    with open(path, 'r', encoding='utf-8') as f:
        text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
        text = text.split('.')
        text = random.sample(text, 20000)
        for sentence in text:
            # add back the fullstop
            sentence += '.'
            m.update(sentence)
    return m

In [ ]:
if __name__ == "__main__":
    start = time.time()
    m = create_ngram_model(6, './voa_fa_2003-2008_orig.txt')

    print (f'Language Model creating time: {time.time() - start}')
    start = time.time()
#     random.seed(44)
    print(f'{"="*50}\nGenerated text:')
    print(m.generate_text(100))
    print(f'{"="*50}')





Language Model creating time: 2.1449925899505615
Generated text:
در زمان صدام، به اين نوع بدرفتاری ها پاداش داده می شد . در ملاقات پرزيدنت بوش و آقای فاکس مسائل مهاجرتی در کانون مذاکرات بود، که همزمان با بحث های سنای آمريکا در مورد لايحه اصلاحات در قانون مهاجرت انجام گرفت . شوکت عزيز به تازگی در پی جلب رای موافق مجلس نمايندگان پاکستان واجد شرايط احراز مقام نخست وزيری شد . ارتش آمريکا نيز می گويد نيروهای ائتلاف امروز در جريان حمله ای در ايالت پکتيا در جنوب شرقی افغانستان مجروح شد . وزيران امورخارجه کشورهای عضو ناتو برای بررسی شماری ازمسائل، ازجمله احتمال عضويت اوکراين، يک اجلاس
